# ДЗ. Тональность отзывов

### Таня Казакова

#### Полезное

In [1]:
import urllib.request
import re
import os
import collections

In [2]:
import nltk

In [3]:
from nltk.tokenize import word_tokenize

In [4]:
from pymystem3 import Mystem

In [5]:
m = Mystem()

## Нужные функции

In [6]:
# Получаем код страницы
def parcer(url):
    page = urllib.request.urlopen(url)
    text = page.read().decode('utf-8')
    return text

In [7]:
# Дополняем словарь "ссылка-оценка" данными с одной страницы
def get_page_linkrating(text, dict_linkrating):
    linkratings_search = re.findall('<meta itemProp="name" content=.+?/><link itemProp="url" href="(.+?)"/>(.|\n)+?<div class="ReviewRating__number">(.+?)</div>', text)
    for tupl in linkratings_search:
        rating = float(re.sub(',', '.', tupl[2]))
        dict_linkrating[tupl[0]] = rating
    return dict_linkrating
###

In [8]:
# Получаем список лемм из отзыва на одной странице
def get_dict_w(review_text):
    tok_text = word_tokenize(review_text)
    words = [w.lower() for w in tok_text if w.isalpha()]
    nlem_text = ' '.join(words)
    lemmas = m.lemmatize(nlem_text)
    dict_page = collections.Counter()
    for lemm in lemmas:
        dict_page[lemm] += 1
    return dict_page

In [9]:
# Получаем множество слов, встречающихся в положительных отзывах
def get_pos_words_set(dict_linkrating):
    pos_words_set = set()
    pos_all_words_dict = collections.Counter()
    for i in dict_linkrating.most_common(140):
        url = i[0]
        review = re.search('<pre class="ReviewContent__text">((.|\n)+?)</pre>', parcer(url))
        review_text = review.group(1)
        dict_page = get_dict_w(review_text)
        pos_all_words_dict = pos_all_words_dict + dict_page
    for word in pos_all_words_dict:
        if pos_all_words_dict[word] > 2:
            pos_words_set.add(word)
    return pos_words_set

In [10]:
# Получаем множество слов, встречающихся в отрицательных отзывах
def get_neg_words_set(dict_linkrating):
    neg_words_set = set()
    neg_all_words_dict = collections.Counter()
    for i in dict_linkrating.most_common()[:-130:-1]:
        url = i[0]
        review = re.search('<pre class="ReviewContent__text">((.|\n)+?)</pre>', parcer(url))
        review_text = review.group(1)
        dict_page = get_dict_w(review_text)
        neg_all_words_dict = neg_all_words_dict + dict_page
    for word in neg_all_words_dict:
        if neg_all_words_dict[word] > 2:
            neg_words_set.add(word)
    return neg_words_set

### Код
Получаем множества только с "позитивными" и только с "негативными" словами

In [11]:
dict_linkrating = collections.Counter()
for i in range(1, 600):
    url_exmpl = 'https://media.auto.ru/reviews/cars/all/?page=NUM&sort=like-desc'
    url = re.sub('NUM', str(i), url_exmpl)
    # print(url)
    text = parcer(url)
    dict_linkrating = get_page_linkrating(text, dict_linkrating)  # Получаем полный словарь "ссылка-оценка"
pos_words_set = get_pos_words_set(dict_linkrating)
neg_words_set = get_neg_words_set(dict_linkrating)
pos_words_doubl_set = pos_words_set
neg_words_doubl_set = neg_words_set
intsect = pos_words_set.intersection(neg_words_set)
pos_words_set.difference_update(intsect)  # Здесь исключительно позитивные
# print(len(pos_words_set))
neg_words_set.difference_update(intsect)  # Здесь исключительно негативные
# print(len(neg_words_set))

In [12]:
# print(len(dict_linkrating.keys())) # Узнать, сколько скачали
# print(dict_linkrating.most_common(140))
# print(dict_linkrating.most_common()[:-130:-1]) # Узнали, сколько взять, чтоб были отрицательные с оценкой <2,5
# Не страшно, что положительных отзывов больше. Положительные чаще на этом сайте бывают короткими.
# Зато у нас будет больше данных хотя бы про положительные.

### Функция, считающая тональность.
1 - положительный отзыв, 0 - отрицательный.

In [13]:
# Даёшь список слов отзыва - получаешь его тональность
def count_mood_rev(check_rev, pos_words_set, neg_words_set):
    pos = 0
    neg = 0
    for word in check_rev:
        if word in pos_words_set:
            pos += 1
        elif word in neg_words_set:
            neg += 1
    sum_pn = pos+neg
    print(pos)
    print(neg)
    print(sum_pn)
    if sum_pn == 0:
        rev_mood = 0.5
    else:
        rev_mood = pos/sum_pn
    return rev_mood

In [18]:
# Это положительные отзывы. rev_mood должен быть ближе к 1 (чем к 0). Проверим работу анализатора на 10 отзывах.
right_mood = 0
false_mood = 0
for i in dict_linkrating.most_common()[140:150]:
    url = i[0]
    review = re.search('<pre class="ReviewContent__text">((.|\n)+?)</pre>', parcer(url))
    review_text = review.group(1)
    dict_page = get_dict_w(review_text)
    check_rev = list(dict_page.elements())
    rev_mood = count_mood_rev(check_rev, pos_words_set, neg_words_set) # Смотрим тональность отзывов
    print(rev_mood)
    print('')
    if rev_mood > 0.65:
        right_mood += 1
    else:
        false_mood += 1
accuracy_p = right_mood/(right_mood+false_mood)
print('Правильность оценки положительных отзывов')
print(accuracy_p)

27
10
37
0.7297297297297297

37
5
42
0.8809523809523809

11
6
17
0.6470588235294118

14
7
21
0.6666666666666666

3
2
5
0.6

15
2
17
0.8823529411764706

6
2
8
0.75

116
22
138
0.8405797101449275

34
9
43
0.7906976744186046

46
22
68
0.6764705882352942

Правильность оценки положительных отзывов
0.8


In [19]:
# Это отрицательные отзывы. rev_mood должен быть ближе к 0 (чем к 1). Проверим работу анализатора на 10 отзывах.
right_mood = 0
false_mood = 0
for i in dict_linkrating.most_common()[-130:-140:-1]:
    url = i[0]
    review = re.search('<pre class="ReviewContent__text">((.|\n)+?)</pre>', parcer(url))
    review_text = review.group(1)
    dict_page = get_dict_w(review_text)
    check_rev = list(dict_page.elements())
    rev_mood = count_mood_rev(check_rev, pos_words_set, neg_words_set) # Смотрим тональность отзывов
    print(rev_mood)
    print(' ')
    if rev_mood < 0.65:
        right_mood += 1
    else:
        false_mood += 1
accuracy_n = right_mood/(right_mood+false_mood)
print('Правильность оценки отрицательных отзывов')
print(accuracy_n)

4
4
8
0.5
 
5
1
6
0.8333333333333334
 
20
11
31
0.6451612903225806
 
7
0
7
1.0
 
29
12
41
0.7073170731707317
 
79
19
98
0.8061224489795918
 
13
2
15
0.8666666666666667
 
12
7
19
0.631578947368421
 
21
10
31
0.6774193548387096
 
28
17
45
0.6222222222222222
 
Правильность оценки отрицательных отзывов
0.4


In [21]:
print('Общая правильность оценки')
print((accuracy_p+accuracy_n)/2)

Общая правильность оценки
0.6000000000000001


#### Идеи о том, как улучшить "анализатор тональности":

 - Увеличить выборку. Скачать гораздо больше отзывов. Проследить, чтоб были крайние позитивные и негативные случаи и чтобы средние не сглаживали картину. В этом конкретном случае найти гораздо больше максимально отрицательных отзывов, чтоб "отрицательных" слов было примерно столько, сколько и положительных.
 - Смотреть скобочки и смайлики. Обращать внимание не только на слова. Скорее всего таких ")" скобочек в положительном отзыве больше, чем открывающихся. Эмоджи тоже нам помогут.
 - Смотреть не только слова, не встречающиеся в противоположном по тональности списке, но и те, у которых частотность существенно в них различается, хотя они встречаются в и положительных, и отрицательных отзывах. Это поможет, наверно, определять тональность для средних/нейтральных отзывов. И стоит рассматривать тональность на бинарно ( + или - ), а как шкалу. И при оценивании считать отклонение от правильной оценки, а не точное попадание в число.
 - Смотреть все отзывы на один продукт, одну модель. (А то вдруг всем нравится LADA и не нравится BMW.) (Хотя тогда получится, что в списке положительного, например, будет двигатель, который всем нравится, а в отрицательных то, что у производителя не вышло.) Поэтому надо смотреть отзывы на разное. Если будем проверять отзывами на ту же модель, хорошо смотреть только на её отзывы. Скорее всего будут жаловаться на то, на что все останые жалуются. И хвалить то же самое. Но гораздо лучше сделать хорошую выборку, где всего в равных количествах много.
 - Можно обучить нейросеть